In [ ]:
from IPython.display import clear_output
!pip install bertopic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
!pip install colorama
!pip install comm

In [ ]:
# Importing dependencies

import os
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import  stopwords
import string
import spacy
import pprint
from colorama import Fore, Back, Style

from bertopic import BERTopic
from bertopic.representation import ZeroShotClassification
from comm import create_comm



# <span style="color:#e74c3c;"> Reading </span> Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
business_data = pd.read_csv('/content/business_data.csv')
entertainment_data = pd.read_csv('/entertainment_data.csv')
sports_data = pd.read_csv('/content/sports_data.csv')
technology_data = pd.read_csv('/content/technology_data.csv')

# merging all data, shuffling and dropping several columns
full_data = pd.concat([business_data, education_data, entertainment_data, sports_data, technology_data], axis = 0)
full_data = full_data.sample(frac = 1).reset_index(drop= True)
full_data.drop(['headlines','description','url'], axis = 1, inplace = True)
full_data.head()

,content,category
0,The Chromebook lineup of notebooks powered by ...,technology
1,Schools in Madhya Pradesh will carry out their...,education
2,You are probably acutely aware of how climate ...,technology
3,The Indian Institute of Management (IIM) Kozhi...,education
4,Artificial intelligence is set to transform th...,technology


# <span style="color:#e74c3c;"> Preprocessing </span>

In [ ]:
%%time
# preprocessing functions
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


CPU times: user 717 ms, sys: 77.9 ms, total: 795 ms
Wall time: 804 ms


In [ ]:
def text_preprocessing(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\\W',' ',text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

In [ ]:

def drop_stopwords(text):
    dropped = [word for word in text.split() if word not in stop_words]
    final_text = ' '.join(dropped)
    return final_text

In [ ]:
def lemmatization(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

In [ ]:

def delete_one_characters(text):
    deleted = [word if len(word)>1 else "" for word in text.split()]
    final_text = ' '.join(deleted)
    return final_text

In [ ]:
def full_preprocessing_pipeline(text):
    text = text_preprocessing(text)
    text = drop_stopwords(text)
    text = lemmatization(text)
    text = delete_one_characters(text)
    return text

data['preprocessed_content'] = data['content'].apply(full_preprocessing_pipeline)

In [ ]:
data.head()

,content,category,preprocessed_content
0,The Chromebook lineup of notebooks powered by ...,technology,chromebook lineup notebook power google empowe...
1,Schools in Madhya Pradesh will carry out their...,education,school madhya pradesh carry academic function ...
2,You are probably acutely aware of how climate ...,technology,probably acutely aware climate change wreak ha...
3,The Indian Institute of Management (IIM) Kozhi...,education,indian institute management iim kozhikode soon...
4,Artificial intelligence is set to transform th...,technology,artificial intelligence set transform workplac...


# <span style="color:#e74c3c;"> BERTopic </span> Model

In [ ]:
# candidate_topics from original dataset
TOPIC_NUM = 10
candidate_topics = ['sports', 'technology','entertainment','education','business']
representation_model = ZeroShotClassification(candidate_topics, model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

topic_model = BERTopic(nr_topics=TOPIC_NUM, zeroshot_min_similarity = 0.7, representation_model=representation_model, verbose=True)

Device set to use cpu


# <span style="color:#e74c3c;"> Training </span>

In [ ]:
topics, probs = topic_model.fit_transform(data['preprocessed_content'].values)

2025-05-05 17:22:55,687 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

2025-05-05 17:35:48,435 - BERTopic - Embedding - Completed ✓
2025-05-05 17:35:48,439 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-05 17:36:40,366 - BERTopic - Dimensionality - Completed ✓
2025-05-05 17:36:40,369 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-05 17:36:40,863 - BERTopic - Cluster - Completed ✓
2025-05-05 17:36:40,864 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-05-05 17:36:42,673 - BERTopic - Representation - Completed ✓
2025-05-05 17:36:42,676 - BERTopic - Topic reduction - Reducing number of topics
2025-05-05 17:36:42,711 - BERTopic - Representation - Fine-tuning topics using representation models.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
2025-05-05 17:37:09,769 - BERTopic - Representation - Completed ✓
2025-05-05 17:37:09,777 - BERTopic - Topic reduction - Reduced

# <span style="color:#e74c3c;"> Topic </span> Results

In [ ]:
# topic no -1 for the outlier topic
# count -> clustering samples

freq = topic_model.get_topic_info()
freq

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3273,-1_say_india_also_film,"[say, india, also, film, indian, sign, year, l...",[end day one series promise plenty thrill twis...
1,0,1535,0_business___,"[business, , , , , , , , , ]",[equity benchmark index sensex nifty close hig...
2,1,1439,1_education___,"[education, , , , , , , , , ]",[student allegedly die suicide kota year city ...
3,2,1152,2_entertainment___,"[entertainment, , , , , , , , , ]",[steady run box office shah rukh khan‘s dunki ...
4,3,1022,3_technology___,"[technology, , , , , , , , , ]",[artificial intelligence make stride across in...
5,4,970,4_sports___,"[sports, , , , , , , , , ]",[former skipper mithali raj feel india ’s chan...
6,5,382,5_sports___,"[sports, , , , , , , , , ]",[tariq panja andrew dasthe premier league accu...
7,6,189,6_space_mission_moon_isro,"[space, mission, moon, isro, crew, launch, lun...",[big year space exploration momentum busy year...
8,7,27,7_sports___,"[sports, , , , , , , , , ]",[russia ’s path send team paris olympics next ...
9,8,11,8_abduljabbar_nba_james_laker,"[abduljabbar, nba, james, laker, lebron, recor...",[lebron james inches close scale summit nba ’s...


In [ ]:
topic_model.visualize_barchart(top_n_topics=TOPIC_NUM, height=200)

# <span style="color:#e74c3c;"> Intertopic Distance </span> Map

In [ ]:
topic_model.visualize_topics()

# <span style="color:#e74c3c;"> Topic Similarity </span> Maxrix

In [ ]:
topic_model.visualize_heatmap(n_clusters=5, top_n_topics=TOPIC_NUM)

# <span style="color:#e74c3c;"> Test Topic Modelling</span> Pipeline

In [ ]:
def create_end2end_topic_prediction(topic_model, df, similarity_threshold:float, top_n_topic:int) -> None:
    """
    @topic_model : BERTopic model
    @df : raw dataframe
    @similarity_threshold : topic similarity threshold
    @top_n_topic : top n topic

    """

    raw_df = df.sample(1)
    raw_df['preprocessed_content'] = raw_df['content'].apply(text_preprocessing).apply(drop_stopwords).apply(lemmatization).apply(delete_one_characters)
    preprocessed_text = raw_df['preprocessed_content'].values[0]

    print('Content :\n'+Fore.BLUE + preprocessed_text + Style.RESET_ALL)
    print('\nOriginal Category : '+Fore.GREEN + raw_df['category'].values[0] + Style.RESET_ALL)

    #calculating topics and similarities
    similar_topics, similarity = topic_model.find_topics(preprocessed_text, top_n=top_n_topic)

    #printing topics
    for i in range(top_n_topic):
        if (similarity[i] >= similarity_threshold) & (similar_topics[i] != -1):
            print(Fore.RED + '\nTopic No: {0} with topic similarity is {1:.5f}'.format(similar_topics[i], similarity[i]) + Style.RESET_ALL)
            print(topic_model.get_topic(similar_topics[i]))
            print("--"*30)

In [ ]:
# random topic prediction
create_end2end_topic_prediction(topic_model, full_data, 0.3, 3)

Content :
you ’re look new tablet budget great deal happen india moment we ’ve round good tablet discount across xiaomi realme oneplus device read detailsredmi padxiaomi ’s affordable redmi pad start rs right micom hdfc bank card user score rs icici customer get rs instant discount via net banking

Original Category : technology

Topic No: 3 with topic similarity is 0.55974
[('technology', 0.905058741569519), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0)]
------------------------------------------------------------

Topic No: 0 with topic similarity is 0.44698
[('business', 0.9860215783119202), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0)]
------------------------------------------------------------


In [ ]:
# random topic prediction
create_end2end_topic_prediction(topic_model, full_data, 0.4, 3)

Content :
kia india tuesday unveil update version midsized sport utility vehicle selto aim garner per cent market share domestic passenger vehicle pv segmentthe automaker currently market share around per cent domestic market also aim double exist sale network plus outlet  selto product enter india since journey kia india selto almost identical segment disruptor segment winner new selto confident take forward strong legacy brand selto lead premium rv market  kia india managing director ceo taejin park saida new spirit rise among uscome forward mesmerize inspire presencethe badass rebornknow selto badassbydesign premiere thenextfromkia thenewseltos thebadassreborn movementthatinspire  kia india kiaind july strategic launch key company ’s ambition attain per cent market share soon addedadvertisement“we feel lot growth potential midsuv segment new selto grow premium end it  park notedalso read kia electric vehicle launch india here ’s much costswith refresh look powerful engine segment ho

In [ ]:
# random topic prediction
create_end2end_topic_prediction(topic_model, full_data, 0.2, 3)

Content :
india ’s oil import set fall eightmonth low july due combination monsoonle fall domestic fuel petroleum product demand low refinery run unit maintenance accord estimate commodity market analytic firm kplerso far july india ’s crude import average million barrel per day bpd around bpd compare june low since november kpler data showedthis story subscriber onlynow subscribe special republic day offer rs nowalready subscriber sign

Original Category : business

Topic No: 0 with topic similarity is 0.60961
[('business', 0.9860215783119202), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0)]
------------------------------------------------------------

Topic No: 1 with topic similarity is 0.41478
[('education', 0.9643800258636475), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0)]
------------------------------------------------------------
